In [2]:
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices(device_type='GPU')


Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
usaPolyGrid = pkl.load(open("util/usaPolyGrid.pkl",'rb'))
base_path = 'Data/'
grids = np.array(list(usaPolyGrid.keys()),np.int32)

## Format Data

In [8]:

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'Data/',
    labels='inferred',
    label_mode = 'categorical',
    color_mode='grayscale',
    batch_size=64,
    image_size=(1000, 1000),
    shuffle=True,
    seed=123,
    validation_split=.15,
    subset="training"
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    'Data/',
    labels='inferred',
    label_mode = 'categorical',
    color_mode='grayscale',
    batch_size=64,
    image_size=(1000, 1000),
    shuffle=True,
    seed=123,
    validation_split=.15,
    subset="validation"
)


KeyboardInterrupt: 

In [10]:
def augment(x,y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return  image,y
ds_train = ds_train.map(augment)


In [23]:
for image, labels in ds_train.take(2):
    print(image.shape)
    print(image[2])

(64, 1000, 1000, 1)
tf.Tensor(
[[[117.96392 ]
  [117.96392 ]
  [117.96392 ]
  ...
  [209.96393 ]
  [209.96393 ]
  [209.96393 ]]

 [[118.42392 ]
  [118.42392 ]
  [118.42392 ]
  ...
  [209.50392 ]
  [209.50392 ]
  [209.50392 ]]

 [[118.96392 ]
  [118.96392 ]
  [118.96392 ]
  ...
  [208.96393 ]
  [208.96393 ]
  [208.96393 ]]

 ...

 [[145.46411 ]
  [142.75014 ]
  [139.45413 ]
  ...
  [ 47.23372 ]
  [ 42.75819 ]
  [ 38.66418 ]]

 [[143.344   ]
  [140.83238 ]
  [137.78397 ]
  ...
  [ 43.251953]
  [ 44.658997]
  [ 45.523617]]

 [[141.96393 ]
  [139.66393 ]
  [136.86392 ]
  ...
  [ 39.663815]
  [ 46.44366 ]
  [ 51.963924]]], shape=(1000, 1000, 1), dtype=float32)


In [11]:
from tensorflow.keras import layers
from tensorflow import keras

model = keras.Sequential([
    layers.Input((1000, 1000, 1)),
    layers.Conv2D(16,3,padding='same'),
    layers.Conv2D(32,3,padding='same'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(10),
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=[
                  keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              ],
              metrics=['accuracy'],
            )
model.fit(ds_train, epochs=10, verbose=2)

Epoch 1/10


In [15]:
dir = 'locations_train.csv'
df = pd.read_csv(dir)

file_paths = df['path'].values
labels = df['grid'].values
training_data =tf.data.Dataset.from_tensor_slices((file_paths, labels))
def read_image(image_file, label):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    return image, label

training_data = training_data.map(read_image).batch(64)

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3), activation='relu'))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(training_data, epochs=10, verbose=2)

Epoch 1/10


ValueError: in user code:

    /home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/shane/Documents/dev/Geoguessr_example/venv/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        r